In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
import matplotlib.pyplot as plt
import numpy as np

# Set publication-quality style
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['figure.titlesize'] = 13

# Color scheme and datasets - MUST BE AT THE TOP
colors = {'CLIP': '#1f77b4', 'CLIP_LoRA': '#ff7f0e', 'Frozen': '#2ca02c'}
datasets_list = ['CIFAR100', 'FOOD101', 'FLOWERS102', 'DTD', 'EUROSAT']

print("Generating all figures...")

# ==================== FIGURE 1: MODEL EFFICIENCY COMPARISON ====================
print("\nGenerating Figure 1: Model Efficiency Comparison...")

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Model data
models = ['CLIP', 'CLIP_LoRA', 'Frozen']
total_params = [151.28, 152.26, 802.78]
trainable_params = [151.28, 0.98, 28.75]
latencies = [274.16, 103.43, 13.35]
memories = [889.00, 777.35, 3342.80]

# Subplot 1: Parameters Comparison
x = np.arange(len(models))
width = 0.35

bars1 = axes[0].bar(x - width/2, total_params, width, label='Total Parameters', 
                     color='lightgray', edgecolor='black', linewidth=1.2, align='center')

# For trainable bars, use minimum visual height for CLIP_LoRA
trainable_visual = trainable_params.copy()
trainable_visual[1] = max(trainable_params[1], 10)  # Minimum 10M for visibility

bars2 = axes[0].bar(x + width/2, trainable_visual, width, label='Trainable Parameters',
                     color='#1f77b4', edgecolor='black', linewidth=1.2, align='center')

axes[0].set_ylabel('Parameters (Million)', fontweight='bold')
axes[0].set_title('(a) Parameter Count Comparison', fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models)
axes[0].legend(loc='upper left', fontsize=8)
axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].set_ylim([-15, 850])

# Add value labels
for bar in bars1:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + 20,
                f'{height:.1f}M',
                ha='center', va='bottom', fontsize=7, fontweight='bold')

for i, (bar, actual_val) in enumerate(zip(bars2, trainable_params)):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + 20,
                f'{actual_val:.2f}M' if i == 1 else f'{actual_val:.1f}M',
                ha='center', va='bottom', fontsize=7, fontweight='bold')

# Subplot 2: Inference Latency
bars = axes[1].bar(models, latencies, color=[colors[m] for m in models],
                   edgecolor='black', linewidth=1.2, align='center')

axes[1].set_ylabel('Average Latency (ms)', fontweight='bold')
axes[1].set_title('(b) Inference Latency', fontweight='bold')
axes[1].grid(axis='y', alpha=0.3, linestyle='--')
axes[1].set_ylim([-5, 320])

for i, (model, lat) in enumerate(zip(models, latencies)):
    axes[1].text(i, lat + 8, f'{lat:.1f}ms',
                ha='center', va='bottom', fontweight='bold', fontsize=7)

# Subplot 3: Memory Usage
bars = axes[2].bar(models, memories, color=[colors[m] for m in models],
                   edgecolor='black', linewidth=1.2, align='center')

axes[2].set_ylabel('Peak GPU Memory (MB)', fontweight='bold')
axes[2].set_title('(c) GPU Memory Consumption', fontweight='bold')
axes[2].grid(axis='y', alpha=0.3, linestyle='--')
axes[2].set_ylim([-50, 3700])

for i, (model, mem) in enumerate(zip(models, memories)):
    axes[2].text(i, mem + 80, f'{mem:.0f}MB',
                ha='center', va='bottom', fontweight='bold', fontsize=7)

plt.tight_layout()
plt.savefig('figure1_efficiency_comparison.png', dpi=300, bbox_inches='tight')
plt.savefig('figure1_efficiency_comparison.pdf', bbox_inches='tight')
print("✓ Figure 1 saved")
plt.close()

# ==================== FIGURE 2: ZERO-SHOT ACCURACY ====================
print("\nGenerating Figure 2: Zero-Shot Classification...")

fig, ax = plt.subplots(figsize=(10, 6))

clip_scores = [62.94, 80.00, 59.98, 40.80, 42.20]
cliplora_scores = [59.07, 77.85, 58.76, 41.49, 52.24]

x = np.arange(len(datasets_list))
width = 0.35

bars1 = ax.bar(x - width/2, clip_scores, width, label='CLIP',
               color='#1f77b4', edgecolor='black', linewidth=1.2)
bars2 = ax.bar(x + width/2, cliplora_scores, width, label='CLIP_LoRA',
               color='#ff7f0e', edgecolor='black', linewidth=1.2)

ax.set_ylabel('Accuracy (%)', fontweight='bold')
ax.set_xlabel('Dataset', fontweight='bold')
ax.set_title('Zero-Shot Classification Performance\n(Frozen: N/A - No Zero-Shot Capability)', 
            fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(datasets_list)
ax.legend(loc='upper right')
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_ylim([0, 100])

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 1,
               f'{height:.1f}',
               ha='center', va='bottom', fontsize=7)

plt.tight_layout()
plt.savefig('figure2_zero_shot_accuracy.png', dpi=300, bbox_inches='tight')
plt.savefig('figure2_zero_shot_accuracy.pdf', bbox_inches='tight')
print("✓ Figure 2 saved")
plt.close()

# ==================== FIGURE 3: LINEAR PROBE ACCURACY ====================
print("\nGenerating Figure 3: Linear Probe Classification...")

fig, ax = plt.subplots(figsize=(10, 6))

clip_linear = [71.78, 84.19, 81.10, 60.05, 89.07]
cliplora_linear = [68.65, 84.04, 83.18, 61.28, 91.35]
frozen_linear = [53.28, 54.53, 49.21, 54.57, 86.59]

x = np.arange(len(datasets_list))
width = 0.25

bars1 = ax.bar(x - width, clip_linear, width, label='CLIP',
               color='#1f77b4', edgecolor='black', linewidth=1.2)
bars2 = ax.bar(x, cliplora_linear, width, label='CLIP_LoRA',
               color='#ff7f0e', edgecolor='black', linewidth=1.2)
bars3 = ax.bar(x + width, frozen_linear, width, label='Frozen',
               color='#2ca02c', edgecolor='black', linewidth=1.2)

ax.set_ylabel('Accuracy (%)', fontweight='bold')
ax.set_xlabel('Dataset', fontweight='bold')
ax.set_title('Linear Probe Classification Performance', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(datasets_list)
ax.legend(loc='upper right')
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_ylim([0, 100])

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 1,
               f'{height:.0f}',
               ha='center', va='bottom', fontsize=7)

plt.tight_layout()
plt.savefig('figure3_linear_probe_accuracy.png', dpi=300, bbox_inches='tight')
plt.savefig('figure3_linear_probe_accuracy.pdf', bbox_inches='tight')
print("✓ Figure 3 saved")
plt.close()

# ==================== FIGURE 4: FEW-SHOT LEARNING CURVES ====================
print("\nGenerating Figure 4: Few-Shot Learning Curves...")

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

datasets_fewshot = {
    'CIFAR100': {
        'CLIP': [33.94, 38.58, 46.21, 50.96, 55.79],
        'CLIP_LoRA': [31.54, 35.79, 43.27, 47.97, 52.05],
        'Frozen': [15.94, 16.90, 21.47, 25.50, 31.29]
    },
    'FOOD101': {
        'CLIP': [40.14, 48.12, 56.04, 61.07, 68.22],
        'CLIP_LoRA': [38.55, 47.82, 56.13, 60.57, 67.23],
        'Frozen': [11.54, 16.14, 20.28, 23.29, 27.70]
    },
    'FLOWERS102': {
        'CLIP': [59.42, 66.61, 75.67, 80.19, 81.10],
        'CLIP_LoRA': [63.65, 71.20, 77.75, 82.86, 83.18],
        'Frozen': [26.25, 33.50, 39.84, 47.60, 49.21]
    },
    'DTD': {
        'CLIP': [35.21, 41.81, 47.66, 52.29, 56.22],
        'CLIP_LoRA': [39.79, 43.88, 47.87, 53.24, 57.23],
        'Frozen': [25.32, 32.98, 39.04, 43.14, 48.35]
    },
    'EUROSAT': {
        'CLIP': [46.72, 46.08, 54.33, 56.06, 61.85],
        'CLIP_LoRA': [64.38, 58.35, 73.89, 70.25, 76.14],
        'Frozen': [43.63, 44.95, 55.17, 56.59, 62.30]
    }
}

shots_x = [1, 2, 4, 8, 16]

for idx, (dataset_name, dataset_data) in enumerate(datasets_fewshot.items()):
    ax = axes[idx]
    
    for model_name in ['CLIP', 'CLIP_LoRA', 'Frozen']:
        ax.plot(shots_x, dataset_data[model_name], 
               marker='o', linewidth=2.5, markersize=8,
               label=model_name, color=colors[model_name])
    
    ax.set_xlabel('Number of Shots per Class', fontweight='bold')
    ax.set_ylabel('Accuracy (%)', fontweight='bold')
    ax.set_title(f'({chr(97+idx)}) {dataset_name}', fontweight='bold')
    ax.set_xscale('log', base=2)
    ax.set_xticks(shots_x)
    ax.set_xticklabels(shots_x)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(loc='best', fontsize=8)
    ax.set_ylim([0, 100])

axes[5].axis('off')

plt.tight_layout()
plt.savefig('figure4_few_shot_learning_curves.png', dpi=300, bbox_inches='tight')
plt.savefig('figure4_few_shot_learning_curves.pdf', bbox_inches='tight')
print("✓ Figure 4 saved")
plt.close()

# ==================== FIGURE 5: PERFORMANCE HEATMAP ====================
print("\nGenerating Figure 5: Performance Heatmap...")

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

zero_shot_data = np.array([
    [62.94, 80.00, 59.98, 40.80, 42.20],
    [59.07, 77.85, 58.76, 41.49, 52.24]
])

linear_probe_data = np.array([
    [71.78, 84.19, 81.10, 60.05, 89.07],
    [68.65, 84.04, 83.18, 61.28, 91.35],
    [53.28, 54.53, 49.21, 54.57, 86.59]
])

few_shot_16_data = np.array([
    [55.79, 68.22, 81.10, 56.22, 61.85],
    [52.05, 67.23, 83.18, 57.23, 76.14],
    [31.29, 27.70, 49.21, 48.35, 62.30]
])

data_matrices = [zero_shot_data, linear_probe_data, few_shot_16_data]
titles = ['(a) Zero-Shot', '(b) Linear Probe', '(c) Few-Shot (16-shot)']
y_labels = [
    ['CLIP', 'CLIP_LoRA'], 
    ['CLIP', 'CLIP_LoRA', 'Frozen'], 
    ['CLIP', 'CLIP_LoRA', 'Frozen']
]

for idx, (ax, data_matrix, title, ylabels) in enumerate(zip(axes, data_matrices, titles, y_labels)):
    im = ax.imshow(data_matrix, cmap='YlOrRd', aspect='auto', vmin=0, vmax=100)
    
    ax.set_xticks(np.arange(len(datasets_list)))
    ax.set_yticks(np.arange(len(ylabels)))
    ax.set_xticklabels(datasets_list)
    ax.set_yticklabels(ylabels)
    
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    for i in range(len(ylabels)):
        for j in range(len(datasets_list)):
            ax.text(j, i, f'{data_matrix[i, j]:.1f}',
                   ha="center", va="center", color="black", 
                   fontweight='bold', fontsize=9)
    
    ax.set_title(title, fontweight='bold', fontsize=12)
    
    if idx == 2:
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label('Accuracy (%)', rotation=270, labelpad=20, fontweight='bold')

plt.tight_layout()
plt.savefig('figure5_performance_heatmap.png', dpi=300, bbox_inches='tight')
plt.savefig('figure5_performance_heatmap.pdf', bbox_inches='tight')
print("✓ Figure 5 saved")
plt.close()

# ==================== FIGURE 6: EFFICIENCY VS PERFORMANCE ====================
print("\nGenerating Figure 6: Efficiency vs Performance Trade-off...")

fig, ax = plt.subplots(figsize=(12, 8))

avg_performance = {
    'CLIP': 66.35,
    'CLIP_LoRA': 67.58,
    'Frozen': 51.71
}

latencies_dict = {'CLIP': 274.16, 'CLIP_LoRA': 103.43, 'Frozen': 13.35}
memories_dict = {'CLIP': 889.00, 'CLIP_LoRA': 777.35, 'Frozen': 3342.80}
trainable_dict = {'CLIP': 151.28, 'CLIP_LoRA': 0.98, 'Frozen': 28.75}
trainable_pct = {'CLIP': 100.0, 'CLIP_LoRA': 0.65, 'Frozen': 3.58}

for model in ['CLIP', 'CLIP_LoRA', 'Frozen']:
    size = memories_dict[model] * 0.8
    
    ax.scatter(latencies_dict[model], avg_performance[model], 
              s=size, alpha=0.5, color=colors[model], 
              edgecolors='black', linewidth=2.5, zorder=3)
    
    offset_x = 15 if model != 'Frozen' else 2
    offset_y = 15 if model == 'CLIP' else (10 if model == 'CLIP_LoRA' else -15)
    
    ax.annotate(model, 
               (latencies_dict[model], avg_performance[model]),
               xytext=(offset_x, offset_y), textcoords='offset points',
               fontsize=13, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.6', facecolor=colors[model], 
                        alpha=0.7, edgecolor='black', linewidth=1.5))
    
    stats = (f'Memory: {memories_dict[model]:.0f} MB\n'
            f'Trainable: {trainable_dict[model]:.2f}M ({trainable_pct[model]:.1f}%)')
    
    ax.annotate(stats, 
               (latencies_dict[model], avg_performance[model]),
               xytext=(offset_x, offset_y - 40), textcoords='offset points',
               fontsize=9, style='italic',
               bbox=dict(boxstyle='round,pad=0.4', facecolor='white', 
                        alpha=0.8, edgecolor='gray', linewidth=0.5))

ax.axhline(y=65, color='green', linestyle='--', linewidth=1.5, alpha=0.5, 
          label='High Performance (>65%)', zorder=1)
ax.axvline(x=150, color='red', linestyle='--', linewidth=1.5, alpha=0.5, 
          label='Real-Time Threshold (<150ms)', zorder=1)
ax.fill_between([0, 150], 65, 75, alpha=0.1, color='green', 
                label='Optimal Region', zorder=1)

ax.set_xlabel('Inference Latency (ms) → Lower is Better', fontweight='bold', fontsize=13)
ax.set_ylabel('Average Performance (%) → Higher is Better', fontweight='bold', fontsize=13)
ax.set_title('Model Efficiency vs Performance Trade-off\n(Bubble size represents GPU memory consumption)', 
            fontweight='bold', fontsize=14)
ax.grid(True, alpha=0.3, linestyle='--', zorder=0)
ax.set_xlim([-20, 300])
ax.set_ylim([36, 72])
ax.legend(loc='lower left', framealpha=0.9, fontsize=10)

ax.text(0.98, 0.02, 
        'CLIP_LoRA: Best efficiency-performance trade-off\n' +
        '✓ Real-time capable (103ms)\n' +
        '✓ Highest accuracy (67.58% avg)\n' +
        '✓ Lowest memory (777MB)\n' +
        '✓ 99.35% parameter reduction',
        transform=ax.transAxes, fontsize=10,
        verticalalignment='bottom', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8, 
                 edgecolor='black', linewidth=1.5))

plt.tight_layout()
plt.savefig('figure6_efficiency_vs_performance.png', dpi=300, bbox_inches='tight')
plt.savefig('figure6_efficiency_vs_performance.pdf', bbox_inches='tight')
print("✓ Figure 6 saved")
plt.close()

print("\n" + "="*80)
print("ALL FIGURES GENERATED SUCCESSFULLY!")
print("="*80)
print("\nGenerated files:")
print("  - figure1_efficiency_comparison.png/.pdf")
print("  - figure2_zero_shot_accuracy.png/.pdf")
print("  - figure3_linear_probe_accuracy.png/.pdf")
print("  - figure4_few_shot_learning_curves.png/.pdf")
print("  - figure5_performance_heatmap.png/.pdf")
print("  - figure6_efficiency_vs_performance.png/.pdf")
print("\nAll figures are publication-ready at 300 DPI!")


Generating all figures...

Generating Figure 1: Model Efficiency Comparison...
✓ Figure 1 saved

Generating Figure 2: Zero-Shot Classification...
✓ Figure 2 saved

Generating Figure 3: Linear Probe Classification...
✓ Figure 3 saved

Generating Figure 4: Few-Shot Learning Curves...
✓ Figure 4 saved

Generating Figure 5: Performance Heatmap...
✓ Figure 5 saved

Generating Figure 6: Efficiency vs Performance Trade-off...
✓ Figure 6 saved

ALL FIGURES GENERATED SUCCESSFULLY!

Generated files:
  - figure1_efficiency_comparison.png/.pdf
  - figure2_zero_shot_accuracy.png/.pdf
  - figure3_linear_probe_accuracy.png/.pdf
  - figure4_few_shot_learning_curves.png/.pdf
  - figure5_performance_heatmap.png/.pdf
  - figure6_efficiency_vs_performance.png/.pdf

All figures are publication-ready at 300 DPI!
